In [1]:
import sys,os
sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
curr_ld_lib_path = ''
os.environ['LD_LIBRARY_PATH'] = curr_ld_lib_path + ':/project/projectdirs/openmsi/jupyterhub_libs/boost_1_55_0/lib' + ':/project/projectdirs/openmsi/jupyterhub_libs/lib'
sys.path.append('/global/project/projectdirs/openmsi/jupyterhub_libs/anaconda/lib/python2.7/site-packages')
sys.path.insert(0,'/project/projectdirs/openmsi/projects/meta-iq/pactolus/pactolus' )

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import Draw
from rdkit.Chem.Draw import MolDrawing
from collections import defaultdict
from matplotlib import pyplot as plt
%matplotlib inline

from rdkit import DataStructs
import numpy as np
import pandas as pd

In [22]:
df = pd.read_csv('Atom.Types.in.SMARTS.csv')
df

,Atom,Functional group,Atom type,Description,Frequency,SMARTS,Comments,Structure used for verification
0,C,Alkane,C1a,R-CH3,16473,C[D1;C],NaN,CC(C=C)CC(C)(C)CC(O)CC=CCO
1,C,Alkane,C1b,R-CH2-R,20193,C[D2;C]C,NaN,CC(C=C)CC(C)(C)CC(O)CC=CCO
2,C,Alkane,C1c,R-CH(-R)-R,4964,C[D3;C](C)C,NaN,CC(C=C)CC(C)(C)CC(O)CC=CCO
3,C,Alkane,C1d,R-C(-R)2-R,698,C[D4;C](C)(C)C,NaN,CC(C=C)CC(C)(C)CC(O)CC=CCO
4,C,Cyclic alkane,C1x,ring-CH2-ring,14010,[C;R][D2;C][C;R],NaN,C1C(C(C(C(O1)))CCCO)O
5,C,Cyclic alkane,C1y,ring-CH(-R)-ring,27376,[C;R][D3;C](C)[C;R],NaN,C1C(C(C(C(O1)))CCCO)O
6,C,Cyclic alkane,C1z,ring-C(-R)2-ring,4463,[C;R][D4;C](C)(C)[C;R],NaN,C1C(C(C(C(O1)))(CCCO)(CCO))O
7,C,Alkene,C2a,R=CH2,634,C=[H2],NaN,CC(C=C)CC(C)(C)CC(O)CC=CCO
8,C,Alkene,C2b,R=CH-R,3965,[!D1;C]=[D2;C],NaN,NCC(CO)=C(CO)CCC(C=C)CC(C)(C)CC(O)CC=CCO
9,C,Alkene,C2c,R=C(-R)2,1914,[!D2;!D1;C]=[C],NaN,NCC(CO)=C(CO)CCC(C=C)CC(C)(C)CC(O)CC=CCO


In [25]:
my_patterns = df['SMARTS'].tolist()
my_molecules = df['Structure used for verification'].tolist()
num_matching = []
for (pat,mol) in zip(my_patterns,my_molecules):
    if type(pat) != float:
        if type(mol) != float:
#             print pat,'####',mol
            matching = Chem.MolFromSmiles(mol).GetSubstructMatches(Chem.MolFromSmarts(pat))
            if len(matching) == 0:
                print pat,'####',mol
                
    


[R;C;D2][R;D2;S][R;D2;S][R;C;D2] #### C1CCCSCCCOCC(=O)1


In [21]:
Chem.MolFromSmiles(mol)
Chem.MolFromSmarts(pat)

In [ ]:
molecule = Chem.MolFromSmiles('C1=CC=C2C(=C1)C(=C(C(=O)O2)CC3=C(C4=CC=CC=C4OC3=O)O)O',True)

fragment = Chem.MolFromSmarts('c[R;c](=O)o')

MolDrawing.elemDict=defaultdict(lambda : (0,0,0))
matching = molecule.GetSubstructMatches(fragment)
print matching
print len(matching)
if len(matching)>0:
    plt.figure(figsize=(20,8*len(matching)))
    for i,f in enumerate(matching):
        plt.subplot(len(matching),2,i+1)
        i1 = Draw.MolToImage(molecule,highlightAtoms=f,highlightColor=[0.6,0.6,1],)
        plt.imshow(i1)
        plt.axis('off')
else:
    plt.figure(figsize=(20,8))
    i1 = Draw.MolToImage(molecule)
    plt.imshow(i1)
    plt.axis('off')

plt.show()